In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# 1. 데이터 준비
# 데이터셋 생성
X, y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)
y = y.reshape(-1, 1)  # y를 2D 형태로 변환

# 데이터 분리 (훈련 데이터 80%, 테스트 데이터 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 표준화
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

# 2. 사용자 정의 층
class MyDenseLayer(Layer):
    def __init__(self, units, **kwargs):
        super(MyDenseLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.weight = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
            name="weight"
        )
        self.bias = self.add_weight(
            shape=(self.units,),
            initializer="zeros",
            trainable=True,
            name="bias"
        )

    def call(self, inputs):
        return tf.nn.relu(tf.matmul(inputs, self.weight) + self.bias)

# 3. 사용자 정의 손실 함수
def huber_loss(y_true, y_pred, delta=1.0):
    error = y_true - y_pred
    is_small_error = tf.abs(error) <= delta
    squared_loss = 0.5 * tf.square(error)
    linear_loss = delta * (tf.abs(error) - 0.5 * delta)
    return tf.where(is_small_error, squared_loss, linear_loss)

# 4. 모델 설계 및 훈련
class CustomModel(Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.dense1 = MyDenseLayer(32)
        self.dense2 = MyDenseLayer(32)
        self.output_layer = MyDenseLayer(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.output_layer(x)

# 모델 생성
model = CustomModel()
model.compile(optimizer=Adam(), loss=lambda y_true, y_pred: huber_loss(y_true, y_pred), metrics=['mse'])

# 모델 훈련
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2)

# 5. 평가 및 예측
# 테스트 데이터에서 MSE 출력
test_loss, test_mse = model.evaluate(X_test, y_test)
print(f"테스트 데이터에서 MSE: {test_mse:.4f}")

# 첫 번째 샘플의 예측값과 실제값 출력
first_sample_pred = model.predict(X_test[:1])
first_sample_true = y_test[:1]
print(f"첫 번째 샘플의 예측값: {scaler_y.inverse_transform(first_sample_pred)[0][0]:.4f}")
print(f"첫 번째 샘플의 실제값: {scaler_y.inverse_transform(first_sample_true)[0][0]:.4f}")


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 0.4334 - mse: 1.0306 - val_loss: 0.3652 - val_mse: 0.8477
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.4179 - mse: 0.9712 - val_loss: 0.3652 - val_mse: 0.8477
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.4476 - mse: 1.0793 - val_loss: 0.3652 - val_mse: 0.8477
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.4540 - mse: 1.0924 - val_loss: 0.3652 - val_mse: 0.8477
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.4710 - mse: 1.1392 - val_loss: 0.3652 - val_mse: 0.8477
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4399 - mse: 1.0442 - val_loss: 0.3652 - val_mse: 0.8477
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.4065 - mse: 0.9496 - val_loss: 0.3652 - val_mse: 0.8477
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.4075 - mse: 0.9529 - val_loss: 0.3652 - val_mse: 0.8477
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.4571 - 